<a href="https://colab.research.google.com/github/basimakram/synthetic-data-generation/blob/main/generate_servicenow_incident_data_with_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ServiceNow Incident Generator with Gemini AI

This notebook automatically generates AWS service incident tickets using Google's Gemini AI and creates them in ServiceNow. Adjust the prompt to make it more realisitic


## 1. Setup and Install Required Packages

In [17]:
# Install required packages
# !pip install google-cloud-aiplatform requests vertexai

## 2. Import Libraries

In [2]:
import requests
import json
import random
import vertexai
from vertexai.generative_models import GenerativeModel
from google.colab import drive
from google.oauth2.service_account import Credentials
import time

print("Libraries imported successfully!")

Libraries imported successfully!


## 3. Mount Google Drive

In [4]:
# Mount Google Drive to access credentials
drive.mount('/content/drive')

Mounted at /content/drive


## 4. Configuration

In [15]:
# ServiceNow credentials - UPDATE THESE
SERVICENOW_USER = ''  # Your ServiceNow username
SERVICENOW_PASSWORD = ''  # Your ServiceNow password
SERVICENOW_URL = 'https://your-servicenow-url.com'  # Your ServiceNow instance URL

# GCP settings - UPDATE THESE
GCP_PROJECT_ID = ''  # Your GCP Project ID
GCP_REGION = 'us-central1'  # GCP region

print("Configuration loaded. Please update your credentials above!")

Configuration loaded. Please update your credentials above!


## 5. Load Google Cloud Credentials

In [7]:
# Load Google credentials from Drive
try:
    with open('/content/drive/MyDrive/google-creds.json') as f:
        gcp_credentials_info = json.load(f)

    gcp_credentials = Credentials.from_service_account_info(
        gcp_credentials_info,
        scopes=['https://www.googleapis.com/auth/cloud-platform']
    )
    print("✅ Google credentials loaded successfully!")

except FileNotFoundError:
    print("❌ google-creds.json not found. Please upload to Google Drive.")
except Exception as e:
    print(f"❌ Error loading credentials: {e}")

✅ Google credentials loaded successfully!


## 6. Get Existing Tickets (Optional)

In [8]:
def get_tickets():
    """Get existing tickets from ServiceNow"""
    url = f'{SERVICENOW_URL}/api/now/table/incident'

    params = {
        'sysparm_display_value': 'true',
        'sysparm_exclude_reference_link': 'true',
        'sysparm_limit': '10'
    }

    try:
        response = requests.get(url, params=params, auth=(SERVICENOW_USER, SERVICENOW_PASSWORD))

        if response.status_code == 200:
            tickets = response.json().get('result', [])
            print(f"✅ Found {len(tickets)} existing tickets")
            return tickets
        else:
            print(f"❌ Failed to get tickets: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ Error: {e}")
        return None


In [9]:
# Test connection (uncomment to run)
tickets = get_tickets()

✅ Found 10 existing tickets


## 7. Initialize Vertex AI

In [10]:
def initialize_vertex_ai():
    """Setup Vertex AI"""
    try:
        vertexai.init(
            project=GCP_PROJECT_ID,
            location=GCP_REGION,
            credentials=gcp_credentials
        )
        print("✅ Vertex AI initialized successfully!")
        return True
    except Exception as e:
        print(f"❌ Error initializing Vertex AI: {e}")
        return False

# Initialize Vertex AI
initialize_vertex_ai()

✅ Vertex AI initialized successfully!


True

## 8. Create Incident Tickets
Adjust the prompt and the required fields accordingly. Check the rest api explorer on SNOW to better understand what or what not to pass

In [11]:
def create_ticket():
    """Generate and create a ServiceNow ticket using AI"""

    # Initialize Gemini model
    model = GenerativeModel("gemini-2.0-flash")

    # AI prompt for generating incident data
    prompt = """Generate a JSON object with:

'impact': random from ['1', '2', '3']
'urgency': random from ['1', '2', '3']
'description': detailed AWS service issue (EC2, Lambda, S3, RDS, DynamoDB, etc.)
'short_description': brief version under 100 characters

Example:
{
    "impact": "2",
    "urgency": "1",
    "short_description": "EC2 instance down in production",
    "description": "Production EC2 instance is unresponsive. SSH connections failing and health checks show failed status. Customer applications are affected. Need immediate investigation."
}

Return only JSON, no extra text."""

    try:
        # Generate with AI
        response = model.generate_content(prompt)
        result = response.text

        # Extract JSON
        start = result.find('{')
        end = result.rfind('}') + 1
        json_content = result[start:end]

        # Parse JSON
        data = json.loads(json_content)

        # Add ServiceNow fields
        subcategories = [
            "EC2 Issues", "Lambda Errors", "S3 Problems",
            "RDS Issues", "API Gateway", "CloudWatch",
            "DynamoDB Performance", "EKS Cluster", "Auto Scaling"
        ]

        data.update({
            "company": "Your Company",
            "caller_id": "Service Account",
            "category": "",
            "subcategory": "",
            "business_service": "AWS Services",
            "assignment_group": "Cloud Team",
            "cmdb_ci": "AWS Production Environment"
        })

        # Create ticket in ServiceNow
        url = f'{SERVICENOW_URL}/api/now/table/incident'
        headers = {"Content-Type": "application/json", "Accept": "application/json"}

        response = requests.post(
            url,
            auth=(SERVICENOW_USER, SERVICENOW_PASSWORD),
            headers=headers,
            json=data
        )

        if response.status_code == 201:
            ticket_number = response.json()['result'].get('number', 'Unknown')
            print(f"✅ Created ticket {ticket_number}")
            print(f"📋 Issue: {data['short_description']}")
            print(f"📂 Category: {data['subcategory']}")
            return True
        else:
            print(f"❌ Failed: {response.status_code}")
            print(f"Error: {response.text}")
            return False

    except Exception as e:
        print(f"❌ Error: {e}")
        return False

print("Ticket creation function defined!")

Ticket creation function defined!


## 9. Create Single Ticket (Test)

In [12]:
# Create single ticket for testing
print("Creating single test ticket...")
create_ticket()

Creating single test ticket...
✅ Created ticket INC0010077
📋 Issue: S3 bucket read errors high
📂 Category: 


True

## 10. Create Multiple Tickets (Batch)

In [13]:
# Create multiple tickets
print("Creating multiple tickets...")
print("=" * 50)

success_count = 0
total_tickets = 2  # Change this number as needed

for i in range(total_tickets):
    print(f"\n🎫 Ticket {i+1}/{total_tickets}:")

    if create_ticket():
        success_count += 1

    # Wait between requests to avoid rate limiting
    if i < total_tickets - 1:
        print("⏳ Waiting 3 seconds...")
        time.sleep(3)

print("\n" + "=" * 50)
print(f"📊 SUMMARY: Created {success_count}/{total_tickets} tickets successfully")
print(f"📈 Success Rate: {(success_count/total_tickets)*100:.1f}%")

Creating multiple tickets...

🎫 Ticket 1/2:
✅ Created ticket INC0010078
📋 Issue: S3 bucket read errors impacting users
📂 Category: 
⏳ Waiting 3 seconds...

🎫 Ticket 2/2:
✅ Created ticket INC0010079
📋 Issue: Lambda function failing due to timeout
📂 Category: 

📊 SUMMARY: Created 2/2 tickets successfully
📈 Success Rate: 100.0%


## 11. Quick Connection Test

In [14]:
# Test all connections quickly
print("🔍 Testing ServiceNow connection...")
test_tickets = get_tickets()

print("\n🔍 Testing Vertex AI...")
try:
    test_model = GenerativeModel("gemini-2.0-flash")
    test_response = test_model.generate_content("Say hello")
    print(f"✅ Vertex AI working: {test_response.text[:50]}...")
except Exception as e:
    print(f"❌ Vertex AI error: {e}")

print("\n🔍 Testing ticket creation...")
create_ticket()

🔍 Testing ServiceNow connection...
✅ Found 10 existing tickets

🔍 Testing Vertex AI...
✅ Vertex AI working: Hello! How can I help you today?
...

🔍 Testing ticket creation...
✅ Created ticket INC0010080
📋 Issue: S3 bucket write errors reported
📂 Category: 


True

## Configuration Checklist

Before running:
- [ ] Update ServiceNow credentials in cell 4
- [ ] Update GCP Project ID in cell 4
- [ ] Upload google-creds.json to Google Drive
- [ ] Run cells 1-7 to setup
- [ ] Test with cell 9 (single ticket)
- [ ] Run cell 10 for batch creation

## Usage Notes

- Change `total_tickets` variable in cell 10 to create more/fewer tickets
- Modify subcategories in cell 8 to match your organization
- Adjust the AI prompt in cell 8 for different incident types
- Use cell 11 to quickly test all connections